# Debugging autoreload

In [ ]:
%load_ext autoreload
%autoreload 2

# Load packages

In [ ]:
from pytorch_tabular.utils import load_covertype_dataset
from rich.pretty import pprint
from plotly.subplots import make_subplots
from pytorch_tabular import TabularModel
import plotly.express as px
import torch
import random
import plotly.graph_objects as go
from scipy import stats
import shap
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.impute import KNNImputer
from glob import glob
import ast
import matplotlib.pyplot as plt
import seaborn as sns
import copy
import itertools
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from pytorch_tabular import model_sweep
from src.pt.model_sweep import model_sweep_custom
import warnings
from src.utils.configs import read_parse_config
from src.pt.hyper_opt import train_hyper_opt
from src.utils.hash import dict_hash
import pathlib
from tqdm import tqdm
import distinctipy
import matplotlib.patheffects as pe
import matplotlib.colors as mcolors
from statannotations.Annotator import Annotator
from scipy.stats import mannwhitneyu
from plottable import ColumnDefinition, Table
from scipy.stats import chi2_contingency
from plottable.plots import bar
from plottable.cmap import normed_cmap, centered_cmap
import optuna
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm
import matplotlib as mpl
from statsmodels.stats.multitest import multipletests
import re
import datetime
from collections import Counter
from matplotlib.ticker import MaxNLocator
from itertools import chain
from sklearn.metrics import mean_absolute_error
from scipy.stats import mannwhitneyu, variation, levene, zscore
import pyaging as pya
import matplotlib.lines as mlines
from src.models.simage.tabular.widedeep.ft_transformer import WDFTTransformerModel
import statsmodels.formula.api as smf
from itertools import chain
from pingouin import ancova
from sklearn.preprocessing import LabelEncoder 
from functools import reduce
import upsetplot
from src.plot.plotly_layout import add_layout
from docx import Document
from docx.shared import Inches, Cm, Mm, Pt
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.enum.section import WD_ORIENT
from docx.enum.style import WD_STYLE_TYPE
from docx.shared import RGBColor
from pathlib import Path
import re
from openai import OpenAI
from matplotlib_venn import venn3, venn3_circles
import functools
from matplotlib_venn import venn2, venn2_circles


def conjunction(conditions):
    return functools.reduce(np.logical_and, conditions)


def disjunction(conditions):
    return functools.reduce(np.logical_or, conditions)

    # Добавление текста с форматированием
    fragments = []
    last_pos = 0
    
    # Разделение текста на фрагменты с форматированием
    for match in bold_re.finditer(text):
        start, end = match.start(), match.end()
        if last_pos < start:
            fragments.append(('normal', text[last_pos:start]))
        fragments.append(('bold', match.group(2)))
        last_pos = end
    
    if last_pos < len(text):
        fragments.append(('normal', text[last_pos:]))
    
    # Обработка курсива внутри оставшихся фрагментов
    final_fragments = []
    for frag_type, content in fragments:
        if frag_type == 'bold':
            final_fragments.append(('bold', content))
            continue
            
        sub_last = 0
        for match in italic_re.finditer(content):
            start, end = match.start(), match.end()
            if sub_last < start:
                final_fragments.append(('normal', content[sub_last:start]))
            final_fragments.append(('italic', match.group(2)))
            sub_last = end
        
        if sub_last < len(content):
            final_fragments.append(('normal', content[sub_last:]))
    
    # Добавление фрагментов в параграф
    for frag_type, content in final_fragments:
        run = paragraph.add_run(content)
        if frag_type == 'bold':
            run.bold = True
        elif frag_type == 'italic':
            run.italic = True


# Mapping for full selection

In [ ]:
path = f"E:/YandexDisk/Work/bbd/fmba/associations/dnam/Special Status/mapping_comparison"
path_draft = "E:/YandexDisk/Work/pydnameth/draft/13_fmba_cvd_dnam/data/120_1"

df_1 = pd.read_excel(f"{path}/2025_05.xlsx", index_col=0)
ctrls_1 = df_1.index[df_1['Special Status'] == 'Control'].tolist()
cases_1 = df_1.index[df_1['Special Status'] == 'Case'].tolist()

df_2 = pd.read_excel(f"{path}/2025_11.xlsx", index_col=0)
ctrls_2 = df_2.index[df_2['Special Status'] == 'Control'].tolist()
cases_2 = df_2.index[df_2['Special Status'] == 'Case'].tolist()

df_ori = pd.read_excel(f"{path_draft}/pheno_funnorm.xlsx", index_col=0)
display(len(df_ori.index.intersection(df_1.index)))

display(list(set(ctrls_1) - set(ctrls_2)))

display(list(set(cases_1) - set(cases_2)))
display(list(set(cases_2) - set(cases_1)))

fig, ax = plt.subplots(figsize=(6, 6))
venn = venn2(
    subsets=(set(ctrls_1), set(ctrls_2)),
    set_labels = ('Old', 'New'),
    set_colors=('chartreuse', 'red'),
    alpha = 0.8
)
venn2_circles(
    subsets=(set(ctrls_1), set(ctrls_2)),
)
plt.tight_layout()
plt.savefig(f"{path}/venn_ctrls.png", bbox_inches='tight', dpi=400)
plt.savefig(f"{path}/venn_ctrls.pdf", bbox_inches='tight', dpi=400)
plt.clf()

fig, ax = plt.subplots(figsize=(6, 6))
venn = venn2(
    subsets=(set(cases_1), set(cases_2)),
    set_labels = ('Old', 'New'),
    set_colors=('chartreuse', 'red'),
    alpha = 0.8
)
venn2_circles(
    subsets=(set(cases_1), set(cases_2)),
)
plt.tight_layout()
plt.savefig(f"{path}/venn_cases.png", bbox_inches='tight', dpi=400)
plt.savefig(f"{path}/venn_cases.pdf", bbox_inches='tight', dpi=400)
plt.clf()


# Subset mapping

In [ ]:
path = f"E:/YandexDisk/Work/bbd/fmba/associations/dnam/Special Status/mapping_comparison"
path_draft = "E:/YandexDisk/Work/pydnameth/draft/13_fmba_cvd_dnam/data/120_1"

df_1 = pd.read_excel(f"{path}/2025_05.xlsx", index_col=0)
df_2 = pd.read_excel(f"{path}/2025_11.xlsx", index_col=0)
df_ori = pd.read_excel(f"{path_draft}/pheno_funnorm.xlsx", index_col=0)

index_cmn = df_2.index.intersection(df_ori.index).values

df_2.loc[index_cmn, 'Special Status Old'] = df_ori.loc[index_cmn, 'Special Status']
df_2.to_excel(f"{path}/ololo.xlsx")